In [22]:
import matplotlib.pyplot as plt
import numpy as np

from skimage import data
from skimage.util import img_as_ubyte
from skimage.filters.rank import entropy
from skimage.morphology import disk
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage import data, color
from skimage.io import imread, imshow
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte
import cv2
from skimage.feature import hog
from skimage import data, exposure
from skimage.feature import greycomatrix, greycoprops
from skimage import data

In [40]:
#Feature Functions
def subsection(image,start_y,start_x,dimension_y,dimension_x):

    s_image = image[start_y:(start_y+dimension_y),start_x:(start_x+dimension_x)]

    return s_image

def get_hog(image):
    fd, hog_image = hog(image, orientations=8, pixels_per_cell=(20, 20),
                    cells_per_block=(2, 2), visualize=True)
    return fd

def get_sift(image):
    sift = cv2.xfeatures2d.SIFT_create()
    kp = sift.detect(image,None)
    return len(kp)
def get_canny(image):
    edges = cv2.Canny(image,50,100)

    return np.count_nonzero(edges)
def get_greycoprops(image):
    g = greycomatrix(image, [1, 2], [0, np.pi/2],
                  normed=True, symmetric=True)
    contrast = greycoprops(g, 'contrast')
    return list(contrast.flatten())         

In [41]:
import csv
import os
import pandas as pd

featuredata = pd.read_csv("features.csv") 
headings = list(featuredata.columns.values)

with open('ImageSubsections.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    count = 1
    lengths = []
    with open("E:/Users/Alex/Eyeballs/300x500WithoutMask.csv", "w",newline='') as fp:
        wr = csv.writer(fp, dialect='excel')
        for row in reader:
            print(count)
            origImageName = row['image']
            start_x  = int(row['start_x'])
            start_y  = int(row['start_y'])
            center_x = int(row['x'])
            center_y = int(row['y'])
            DR       = row['exudates'] 
            imageName = origImageName.strip('ODmasked-')
            fileLocation = 'E:/Users/Alex/Dropbox/MEng Software/Winter-Term-2019/Big Data/Project/original cropped/' +imageName


            if os.path.isfile(fileLocation):
                im = imread(fileLocation)

                imgrey = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

                #dimension of subsection
                dimension_x = 300 #dimension of subsection
                dimension_y = 500 #dimension of subsection

                if start_x+dimension_x > 1300:
                        start_x = 1300-dimension_x
                if start_y+dimension_y > 1000:
                    start_y = 1000-dimension_y
                imgrey_sub = subsection(imgrey,start_y,start_x,dimension_y,dimension_x)
                im_sub = subsection(im,start_y,start_x,dimension_y,dimension_x)
    #            print(start_x,im_sub.shape)

                hog_values = list(get_hog(im_sub))
                sift_kps = get_sift(im_sub)
                canny_count = get_canny(im_sub)
                gcp_values = get_greycoprops(imgrey_sub)

                if count == 1:
                    headings += ["hog"+str(i) for i in range(1,len(hog_values)+1)]
                    headings += ["siftkps"]
                    headings += ["cannycount"]
                    headings += ["greycoprops"+str(i) for i in range(1,len(gcp_values)+1)]
                    wr.writerow(headings)

                row_list = list(featuredata[featuredata["image"]==origImageName].values[0])
                print(row_list)
                if len(row_list) != 0:
                    row_list += hog_values
                    row_list += [sift_kps]
                    row_list += [canny_count]
                    row_list += gcp_values
                    count +=1

                    if len(headings)!=len(row_list):
                        print("error at: ",imageName,count)
                        print(im_sub.shape)
                        break
                    wr.writerow(row_list)
        #             if len(row_list) not in lengths:
        #                 print(len(row_list))
        #                 lengths.append(len(row_list)) 

            #wr.writerow(row_list)    


1


C:\Users\Alex\Anaconda3\lib\site-packages\skimage\feature\_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)


['ODmasked-image-1.png', 4.388435708063292, 0, 9604, 311.44983544705883, 1129.95752132547, 742.2458472714176, 0]
2
['ODmasked-image-10.png', 4.252627417407548, 1, 0, 0.0, 0.0, 0.0, 0]
3
['ODmasked-image-100.png', 3.498504735537331, 0, 0, 0.0, 0.0, 0.0, 1]
4
4
['ODmasked-image-1001.png', 3.661230583446621, 0, 76, 287.862814548875, 466.80402740336336, 385.75223579162025, 0]
5
5
5
5
['ODmasked-image-1005.png', 3.3337463547817285, 0, 162, 278.44209451877066, 464.17453613915535, 379.4265140811924, 1]
6
['ODmasked-image-1006.png', 3.389824500825614, 1, 1220, 315.3363283860583, 431.5854492449902, 342.34870126875694, 1]
7
['ODmasked-image-1007.png', 3.605727036543596, 0, 3606, 189.76037521042164, 450.2110616144388, 226.75324343581843, 0]
8
8
8
['ODmasked-image-101.png', 4.159030485619573, 1, 0, 0.0, 0.0, 0.0, 1]
9
9
['ODmasked-image-1011.png', 3.7992617460540297, 0, 384, 148.0033783398203, 714.5299153989287, 458.2404510063163, 1]
10
['ODmasked-image-1012.png', 3.7054140181836415, 1, 278, 247.1

['ODmasked-image-1088.png', 3.595578636421907, 1, 202, 255.92381678929374, 561.4445653846869, 462.25193314299247, 0]
75
75
['ODmasked-image-109.png', 4.2245943440396925, 1, 3028, 123.40583454602137, 884.1549637931123, 637.477811690829, 1]
76
['ODmasked-image-1090.png', 3.672916208144848, 1, 206, 396.07322555305353, 524.2146506918707, 469.0411107038015, 0]
77
77
['ODmasked-image-1092.png', 3.7607238661719538, 1, 608, 359.6109008358896, 542.0590373750815, 460.32268723356316, 0]
78
['ODmasked-image-1093.png', 3.210046966545732, 0, 8, 513.5153357008921, 581.3518727930615, 563.6483689405803, 0]
79
['ODmasked-image-1094.png', 3.411431557455733, 1, 250, 397.77506206397607, 553.7011829497928, 487.07677798250916, 0]
80
['ODmasked-image-1095.png', 3.224298288156916, 0, 628, 307.31905245200795, 992.0408257728106, 644.4921214588568, 1]
81
['ODmasked-image-1096.png', 3.4625968960767413, 1, 236, 60.083275543199214, 575.6961003863063, 525.2191932215771, 1]
82
['ODmasked-image-1097.png', 3.46433890706

['ODmasked-image-1162.png', 3.6277886806694983, 1, 242, 252.2399651125888, 570.7337382703076, 498.7558726802634, 1]
145
['ODmasked-image-1163.png', 3.31087718250776, 0, 224, 273.40811985016103, 471.07111989592397, 373.7541710878388, 0]
146
['ODmasked-image-1164.png', 3.815349223823564, 1, 18, 247.92337525937324, 580.2689031819643, 497.4368875091957, 0]
147
['ODmasked-image-1165.png', 3.497374916892987, 0, 2520, 158.14550262337528, 440.78339351658883, 193.8131227919986, 0]
148
['ODmasked-image-1166.png', 3.893093931888739, 1, 888, 241.66091947189145, 579.4350697015153, 494.6178713825708, 0]
149
['ODmasked-image-1167.png', 3.2452600308180743, 0, 166, 293.7839342101606, 456.018639969903, 368.6618038105221, 0]
150
['ODmasked-image-1168.png', 3.3894430026751454, 1, 140, 395.83961398526037, 587.3346575845835, 509.144415114287, 1]
151
['ODmasked-image-1169.png', 4.08569661430481, 0, 134, 311.01446911679204, 1025.7504569825937, 587.3688715044983, 0]
152
['ODmasked-image-117.png', 3.53832790899

...

['ODmasked-image-666.png', 3.2915991606311814, 0, 3866, 304.3451987464235, 997.0080240399272, 627.0395346777992, 1]
1079
['ODmasked-image-667.png', 3.1888522525194563, 0, 742, 338.49667649771686, 588.1292374980179, 407.5856738506436, 1]
1080
['ODmasked-image-668.png', 3.6061705412710183, 1, 524, 206.30559856678641, 594.7839944046914, 447.535898738289, 1]
1081
['ODmasked-image-669.png', 2.9738991085193107, 0, 0, 0.0, 0.0, 0.0, 1]
1082
['ODmasked-image-67.png', 4.2860564880941965, 1, 0, 0.0, 0.0, 0.0, 0]
1083
['ODmasked-image-670.png', 3.4396575889351, 1, 1590, 379.1530034168264, 553.7661961514083, 475.35530501057565, 1]
1084
['ODmasked-image-671.png', 2.822136313768049, 1, 3078, 73.78346698278686, 426.41411796515365, 383.1209178219663, 1]
1085
['ODmasked-image-672.png', 3.124846231906455, 1, 0, 0.0, 0.0, 0.0, 1]
1086
['ODmasked-image-673.png', 3.1896533838898877, 0, 712, 278.792037189013, 471.1698207652948, 371.85411939589113, 1]
1087
['ODmasked-image-674.png', 2.9738991085193107, 0, 0,

['ODmasked-image-731.png', 3.5874909044369514, 1, 676, 215.59452683219956, 604.4145928086117, 483.5415868140531, 0]
1152
['ODmasked-image-732.png', 3.676696161018088, 0, 156, 554.3978715687858, 993.227567076146, 823.2818100454166, 0]
1153
['ODmasked-image-733.png', 3.2269136509981644, 0, 5442, 259.3838853899756, 472.2298169323915, 409.9435817314756, 0]
1154
['ODmasked-image-734.png', 3.2363498794601884, 1, 24, 456.1326561429252, 588.7113044608537, 525.4331832023598, 0]
1155
['ODmasked-image-735.png', 3.2626832877081697, 0, 924, 293.218348675522, 468.5466892423849, 369.22838970212564, 0]
1156
['ODmasked-image-736.png', 3.441384103259768, 0, 3578, 184.0108692441835, 481.4810484328537, 310.22316520970463, 0]
1157
['ODmasked-image-737.png', 3.482552354534017, 1, 822, 251.80349481291958, 559.3433650272433, 472.2338674643168, 0]
1158
['ODmasked-image-738.png', 3.3467654186289733, 0, 1734, 307.93830550939913, 988.716845208981, 567.0740982375834, 0]
1159
['ODmasked-image-739.png', 3.2727483645

['ODmasked-image-796.png', 3.5259554963348347, 0, 804, 308.15093704222284, 1001.8028748211896, 652.5759354490674, 0]
1223
['ODmasked-image-797.png', 3.3676555209616508, 1, 6196, 110.47624178980746, 612.1805289291714, 308.0663355223276, 0]
1224
['ODmasked-image-798.png', 3.539619109171877, 0, 964, 303.64617567161946, 1004.8203819588852, 639.1951080073559, 0]
1225
['ODmasked-image-799.png', 3.542489838073882, 0, 1000, 304.80321520613916, 1001.8028748211896, 636.107522799096, 0]
1226
['ODmasked-image-8.png', 3.8641689359254863, 1, 0, 0.0, 0.0, 0.0, 0]
1227
['ODmasked-image-80.png', 3.5742603725452264, 1, 0, 0.0, 0.0, 0.0, 0]
1228
['ODmasked-image-800.png', 3.330632217712798, 1, 6244, 72.13875518748573, 581.4894668005604, 445.92942027851507, 0]
1229
['ODmasked-image-801.png', 3.3204510264618636, 0, 332, 290.66303514551004, 472.45105566608703, 386.5604688378466, 0]
1230
['ODmasked-image-802.png', 3.507693520520029, 1, 1058, 374.7705964987115, 557.9435455312661, 474.2729858491559, 0]
1231
['

['ODmasked-image-86.png', 3.84383951610775, 0, 0, 0.0, 0.0, 0.0, 1]
1293
['ODmasked-image-860.png', 3.367556679985371, 1, 950, 238.933045014707, 572.8420375635853, 443.4562982710872, 1]
1294
['ODmasked-image-861.png', 3.4117579590985505, 0, 1382, 311.96954979612997, 979.9775507632816, 674.4136113743891, 1]
1295
['ODmasked-image-862.png', 3.3905268305844314, 0, 682, 268.88845270855353, 487.68945036775193, 378.73556494601195, 1]
1296
['ODmasked-image-863.png', 3.4818093191322474, 1, 724, 420.0761835667431, 559.3433650272433, 501.22364164219215, 1]
1297
['ODmasked-image-864.png', 3.1233111257432435, 0, 900, 297.9295218671691, 473.83119357003085, 383.4126357512349, 1]
1298
['ODmasked-image-865.png', 3.484402541103535, 1, 2318, 74.65252842335615, 107.37783756436892, 92.47946751237926, 1]
1299
['ODmasked-image-866.png', 3.3181378885583834, 0, 1408, 277.2760357477725, 488.26222462934817, 385.8078151939257, 1]
1300
['ODmasked-image-867.png', 3.3823133108378514, 0, 1318, 315.03968004046726, 982

['ODmasked-image-938.png', 3.283222837082287, 0, 180, 292.5149568825499, 472.76209661943085, 383.9322910815416, 0]
1363
['ODmasked-image-939.png', 3.373760783702062, 1, 8, 249.61770770520266, 350.96011169362254, 277.9584275150433, 0]
1364
['ODmasked-image-94.png', 4.166945461992332, 1, 806, 270.49029557453633, 605.1784860683664, 535.577557054546, 1]
1365
1365
1365
['ODmasked-image-942.png', 2.9996359552921383, 0, 184, 221.41589825484527, 465.45031958308937, 347.60216816378113, 1]
1366
['ODmasked-image-943.png', 3.292118641682105, 1, 718, 380.38007308480286, 550.3108212637654, 473.88885547136863, 1]
1367
['ODmasked-image-944.png', 3.0188106676733577, 0, 358, 153.8440769090575, 462.7158955557935, 307.78334467901726, 1]
1368
['ODmasked-image-945.png', 3.3191639335419785, 1, 652, 250.79872407968904, 564.2729835815286, 463.7006160888303, 1]
1369
1369
1369
['ODmasked-image-948.png', 3.185699826277303, 0, 122, 281.1565400270817, 460.64519969277876, 367.9303163112489, 1]
1370
['ODmasked-image-